<a href="https://colab.research.google.com/github/JoseVictorio/GRD_MAPATON/blob/main/Algoritmo_para_Conteo_de_P%C3%A9rdidas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Algoritmo para Conteo de Pérdidas

In [ ]:
# ---------------------------------------------------------------------------------- #
# --------------------- Algoritmo para Conteo de Pérdidas -------------------------- #
# ---------------------------------- A.C.P. ---------------------------------------- #
# ---------- Asistente para la Gestión del Riesgo en Estado de Emergencia ---------- #
# ----------------------------- Mapatón Perú 2021 ---------------------------------- #

# ---------------------------------------------------------------------------------- #

Explicación:
- Posterior a la ocurrencia de un desastre, es importante que se adopten medidas para 
cuantificar los daños causados, es por ello, que se pretende el desarrollo de un 
algoritmo que realice este proceso, automatizando las cifras según un criterio de
muestras y algoritmos de aprendizaje autónomo.

Los algoritmos utilizados pertenecen a los empleados por el machine learning.

# ---------------------------------------------------------------------------------- #

ACP nace para ayudar al conteo posterior de la ocurrencia de un desastre. Para su 
correcto manejo, la elección de las muestras debe ser realizada por un fotointerprete
experimentado, además, la imagen a trabajar debe contar con el formato .tiff. La
última apreciación puede ser variada según la fuente de los datos que se maneje (Esto
demuestra su escabilidad y replicabilidad).

# ---------------------------------------------------------------------------------- #

In [ ]:
# ----------------------------------------------------------------------------- #
#                             ASISTENTE A.C.P.                                  #
# ----------------------------------------------------------------------------- #
#   Desarrollado como apoyo a la Gestión del Riesgo en el transcurso de la      #
#                             Mapatón Perú 2021                                 #
# ----------------------------------------------------------------------------- #
#                        Equipo: Nombre del equipo                              #
# ----------------------------------------------------------------------------- #
#                                                                               #
# --------------------------- Nota aclaratoria -------------------------------- #
#                                                                               #
#     Para la realización del código expuesto posteriormente se hizo uso de     #
#   imágenes ejemplificadoras, con el fin, de evidenciar la funcionalidad de    #
#                  este y, su actuación en un evento real.                      #
#                                                                               #
# ----------------------------------------------------------------------------- #



# --------------------------- Variables previas ------------------------------- #

%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.plot import show
from skimage.feature import match_template
import numpy as np
from PIL import Image
from sklearn.cluster import Birch, MiniBatchKMeans

# ------------------- Generación de funcionalidades -------------------------- #

puntos = gpd.read_file('Muestras/Muestras.shp')
print('CRS de las muestras: ' + str(puntos.crs))

imagen = rasterio.open('Rst/Mosaico.tif')
print('CRS de la imagen: ' + str(imagen.crs))
print('Número de bandas de la imagen: ' + str(imagen.count))

fig, ax = plt.subplots(figsize=(10,10))
puntos.plot(ax=ax, color='blue', marker='+', markersize=50)
show(imagen, ax=ax)

banda = imagen.read(3) # Elección de la banda a utilizar, se recomienda el uso de la banda térmica para mayor efectividad.

coordenadas = []

for index, values in puntos.iterrows():
    x = values['geometry'].xy[0][0]
    y = values['geometry'].xy[1][0]
    row, col = imagen.index(x,y)
    print("Punto N°:%d corresponde a la fila: %d, y columna: %d"%(index,row,col))
    coordenadas.append([row,col])

print('Número de muestras: %d'%len(coordenadas))

radio = 15 # Variar el radio de búsqueda para mejores resultados según el caso
    
# ----------------------------------------------------------------------------- #
# -------------------- Ejecución y prueba del algoritmo ----------------------- #

fig, ax = plt.subplots(1, len(coordenadas),figsize=(20,20))

for index, item in enumerate(coordenadas):
    row = item[0]
    col = item[1]
    ax[index].imshow(banda)
    ax[index].plot(col, row, color='red', linestyle='dashed', marker='+',
     markerfacecolor='blue', markersize=8)
    ax[index].set_xlim(col-radio,col+radio)
    ax[index].set_ylim(row-radio,row+radio)
    ax[index].axis('off')
    ax[index].set_title(index)

listaresultados = []

templateBandList = []

for rowCol in coordenadas:
    imageList = []
    row = rowCol[0]
    col = rowCol[1]
    
    imageList.append(banda[row-radio:row+radio, col-radio:col+radio])
    
    templateBandToRotate = banda[row-2*radio:row+2*radio, col-2*radio:col+2*radio]
    rotationList = [i*3 for i in range(0,4)]
    for rotation in rotationList:
        rotatedRaw = Image.fromarray(templateBandToRotate)
        rotatedImage = rotatedRaw.rotate(rotation)
        imageList.append(np.asarray(rotatedImage)[radio:-radio,radio:-radio])
    
    fig, ax = plt.subplots(1, len(imageList),figsize=(12,12))
    for index, item in enumerate(imageList):
        ax[index].imshow(imageList[index])
    
    templateBandList+=imageList

matchXYList = []

for index, templateband in enumerate(templateBandList):
    if index%10 == 0:
        print('Coincidencias en la imagen Nº %d'%index)
    matchTemplate = match_template(banda, templateband, pad_input=True)
    matchTemplateFiltered = np.where(matchTemplate > np.quantile(matchTemplate,0.999999)) # Aproximación al 99.99999 % de parecido con el entorno de las muestras 
    for item in zip(matchTemplateFiltered[0],matchTemplateFiltered[1]):
        x, y = imagen.xy(item[0], item[1])
        matchXYList.append([x,y])

fig, ax = plt.subplots(figsize=(10, 10))

matchXYArray = np.array(matchXYList)

ax.scatter(matchXYArray[:,0],matchXYArray[:,1], marker='+',c='orangered', s=100, alpha=0.5)

show(imagen, ax=ax)

# Aplicación de algoritmos de aprendizaje autonomo 

brc = Birch(branching_factor=10000, n_clusters=None, threshold=0.0001, compute_labels=True)

brc_2 = MiniBatchKMeans(init='k-means++', n_clusters=100, batch_size=100,
                      n_init=10, max_no_improvement=10, verbose=0,
                      random_state=0)

brc.fit(matchXYArray)

brc_2.fit(matchXYArray)

birchPoint_2 = brc_2.cluster_centers_
birchPoint_2

birchPoint = brc.subcluster_centers_
birchPoint

fig = plt.figure(figsize=(7, 9))

ax = fig.add_subplot(111)

ax.scatter(birchPoint_2[:,[0]],birchPoint_2[:,[1]], marker='+',color='red',s=100, alpha=1) # Según el entorno de estudio, se cambia la variable resultante del algoritmo de aprendizaje

show(imagen, ax=ax)

plt.show()

np.savetxt("Resultados/Resultado.csv", birchPoint, delimiter=",")

# ----------------------------- Fin del código -------------------------------- #
# ----------------------------------------------------------------------------- #